In [2]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.7.1" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia CSV DataFrames Random Statistics Distributions LinearAlgebra SparseArrays Printf JWAS"  # CSV DataFrames Random Statistics JWAS Distributions LinearAlgebra
JULIA_PACKAGES_IF_GPU=""
JULIA_NUM_THREADS=4
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Unrecognized magic `%%shell`.

Julia does not use the IPython `%magic` syntax.   To interact with the IJulia kernel, use `IJulia.somefunction(...)`, for example.  Julia macros, string macros, and functions can be used to accomplish most of the other functionalities of IPython magics.


In [3]:
versioninfo()

Julia Version 1.7.1
Commit ac5cc99908 (2021-12-22 19:35 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-12.0.1 (ORCJIT, broadwell)
Environment:
  JULIA_NUM_THREADS = 4


Run basic julia from https://ebook.qtl.rocks/julia/julialanguage

In [4]:
a = 2.5
5a
# this is a one line comment
9a # comment can be at the end of a line too

22.5

#= This is the first line of a comment block
9a # comment can be at the end of a line too
This is the last line of the block =# 


In [5]:
a,b = 1,2
[a,b]

2-element Vector{Int64}:
 1
 2

In [6]:
# easy swapping of values
a,b = b,a
[a,b]

2-element Vector{Int64}:
 2
 1

1.1.3. Julia allows simple boolean expressions

In [7]:
# Example
1 < a < 3
# In many languages this has to be written as
1 < a && a < 3

true

In [8]:
a = 1
b = 1
c = 2

res = a==b ? a : b    # if a=b, rest is equal to a and else is b

1

In [9]:
a==c ? a : c

2

1.1.5. Remainder

In [10]:
99 % 10


9

Arrays (Vectors and Matricies)

In [11]:
# empty vector of double precision floats
v = Array{Float64,1}()   #dimension 1, float64 is to be precision memory state or 64 bits

Float64[]

In [12]:
push!(v,2.0)   # push a number to v or assigning a value to v
v

1-element Vector{Float64}:
 2.0

In [13]:
push!(v,3.6)   # push a number to v or assigning a value to v
v  # now 2 elements

2-element Vector{Float64}:
 2.0
 3.6

In [14]:
# vector of size 5, double precision floats 64 bits (32+32 bits)
u =  Array{Float64,1}(undef, 5)   #random, undefined values

5-element Vector{Float64}:
 0.0
 6.9297812628985e-310
 6.9297812575879e-310
 0.0
 6.92979205092703e-310

In [15]:
# matrix of size 3x5, double precision floats
U =  Array{Float64,2}(undef, 3,5)   #random, undefined values

3×5 Matrix{Float64}:
 5.0e-324  0.0       0.0       5.0e-324  5.0e-324
 2.0e-323  5.0e-324  5.0e-324  0.0       0.0
 5.0e-324  5.0e-324  0.0       0.0       6.92978e-310

In [16]:
# vector of size 5, of 2.2
u =  fill(2.2,5)    #fill is to repeat

5-element Vector{Float64}:
 2.2
 2.2
 2.2
 2.2
 2.2

In [17]:
zeros(5)   # repeat zero 5 times
zeros(3,5)  #matrix 3*2
zero(U)  #zero to U matrix

3×5 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

In [18]:
using Random, Statistics, Distributions, LinearAlgebra, DataFrames, StatsPlots

LoadError: ignored

In [19]:
using Pkg

In [20]:
Pkg.add("StatsPlots")  #was missing

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed JpegTurbo_jll ──────────────── v2.1.2+0
   Installed x265_jll ───────────────────── v3.5.0+0
   Installed libfdk_aac_jll ─────────────── v2.0.2+0
   Installed GR_jll ─────────────────────── v0.66.0+0
   Installed Libmount_jll ───────────────── v2.35.0+0
   Installed LERC_jll ───────────────────── v3.0.0+1
   Installed FFTW ───────────────────────── v1.5.0
   Installed OffsetArrays ───────────────── v1.12.7
   Installed LoggingExtras ──────────────── v0.4.9
   Installed Opus_jll ───────────────────── v1.3.2+0
   Installed RelocatableFolders ─────────── v0.3.0
   Installed Xorg_xkbcomp_jll ───────────── v1.4.2+4
   Installed Measures ───────────────────── v0.3.1
   Installed Contour ────────────────────── v0.6.2
   Installed EarCut_jll ─────────────────── v2.2.3+0
   Installed Grisu ──────────────────────── v1.0.2
   Installed Xorg_xcb_util_wm_jll ───────── v0.4.1+1
   Installed Xorg

In [23]:
Random.seed!(101) #set seed
rand(3,5) # uniform(0,1), default 0 to 1
randn(3,5) # standard normal

3×5 Matrix{Float64}:
  1.92234     0.308766  -0.250012  -0.156213  -0.566148
 -0.0976424  -0.514017   0.868958   1.94966   -1.88439
  0.145262    0.111226   0.361192   1.00426   -0.21067

In [24]:
mean(randn(1_000_000)) #underscore rather than , 1,000,000

-8.252374275052659e-5

In [25]:
var(randn(1_000_000))

0.9996430905257744

In [26]:
M = rand(Binomial(2,0.5),10,20)  #n=2, p=0.5 probability of success 0-2

10×20 Matrix{Int64}:
 0  1  1  1  1  2  0  1  1  1  1  1  2  1  0  0  1  0  0  2
 0  1  0  1  1  0  1  0  1  1  2  0  2  1  1  1  2  1  2  1
 0  1  0  1  1  0  1  0  2  2  0  2  0  0  0  1  2  0  1  1
 0  1  0  1  1  2  1  1  1  2  0  1  1  1  1  1  0  1  2  0
 2  1  2  1  1  2  2  0  1  2  1  0  2  2  2  1  1  1  1  1
 0  1  0  0  0  2  1  2  1  1  1  0  1  0  1  1  1  1  1  1
 2  2  1  2  1  0  2  1  0  1  1  1  1  1  1  1  0  1  1  0
 0  1  1  0  2  0  2  0  0  1  1  0  2  2  2  2  1  2  2  1
 1  0  1  0  2  1  1  0  0  2  2  2  0  2  2  0  1  1  1  1
 1  1  1  2  1  1  1  1  0  1  1  1  1  1  1  1  1  1  1  1

In [27]:
M[:,1]  #all rows and 1 column

10-element Vector{Int64}:
 0
 0
 0
 0
 2
 0
 2
 0
 1
 1

In [28]:
M[1,:] #row

20-element Vector{Int64}:
 0
 1
 1
 1
 1
 2
 0
 1
 1
 1
 1
 1
 2
 1
 0
 0
 1
 0
 0
 2

Transposing and array gives a row vector

In [29]:
M[1,:]'

1×20 adjoint(::Vector{Int64}) with eltype Int64:
 0  1  1  1  1  2  0  1  1  1  1  1  2  1  0  0  1  0  0  2

In [30]:
M[1,:]'M[1,:] #Sum of squares of elements in first row
M[1,:]'*M[1,:]
dot(M[1,:],M[1,:])
M[1,:].^2 #manual dot product
sum(M[1,:].^2)

23

Matrix algebra pkg

In [31]:
1I(3)   #1 in diagonals for 3*3


3×3 Diagonal{Int64, Vector{Int64}}:
 1  ⋅  ⋅
 ⋅  1  ⋅
 ⋅  ⋅  1

In [32]:
A = M'M + I   # assign two matrix
B = rand(3,3) + I

3×3 Matrix{Float64}:
 1.63234   0.548128  0.615056
 0.10876   1.48163   0.0390292
 0.632732  0.637836  1.01819

In [ ]:
B == B*I  #same dimension matrix
B === B*I



Diagonal Matrix from

In [ ]:
Diagonal(A)

In [ ]:
v = [1,2,3]
Diagonal(v)

In [ ]:
B = A[1:2,3:4]   #subset a matrix

In [ ]:
C = A[1:2,11:end]  #end to end

In [ ]:
hcat(B,C)   #bind rows of two matrix horizen
[ 
A[1:2,:]
A[11:end,:]
]
vcat(A[1:2,:],A[11:end,:])     #bind columns of two matrix vert


Sorting

In [33]:
sort(M[:,1]) #by first column

10-element Vector{Int64}:
 0
 0
 0
 0
 0
 0
 1
 1
 2
 2

In [34]:
index = sortperm(M[:,1])
M[index,1]

10-element Vector{Int64}:
 0
 0
 0
 0
 0
 0
 1
 1
 2
 2

Sorting the entire matrix by the first column

In [35]:
M[index,:]

10×20 Matrix{Int64}:
 0  1  1  1  1  2  0  1  1  1  1  1  2  1  0  0  1  0  0  2
 0  1  0  1  1  0  1  0  1  1  2  0  2  1  1  1  2  1  2  1
 0  1  0  1  1  0  1  0  2  2  0  2  0  0  0  1  2  0  1  1
 0  1  0  1  1  2  1  1  1  2  0  1  1  1  1  1  0  1  2  0
 0  1  0  0  0  2  1  2  1  1  1  0  1  0  1  1  1  1  1  1
 0  1  1  0  2  0  2  0  0  1  1  0  2  2  2  2  1  2  2  1
 1  0  1  0  2  1  1  0  0  2  2  2  0  2  2  0  1  1  1  1
 1  1  1  2  1  1  1  1  0  1  1  1  1  1  1  1  1  1  1  1
 2  1  2  1  1  2  2  0  1  2  1  0  2  2  2  1  1  1  1  1
 2  2  1  2  1  0  2  1  0  1  1  1  1  1  1  1  0  1  1  0

Sorting matrix by the sum of columns 1 and 2

In [36]:
M[sortperm( M[:,1]+M[:,2] ),:]

10×20 Matrix{Int64}:
 0  1  1  1  1  2  0  1  1  1  1  1  2  1  0  0  1  0  0  2
 0  1  0  1  1  0  1  0  1  1  2  0  2  1  1  1  2  1  2  1
 0  1  0  1  1  0  1  0  2  2  0  2  0  0  0  1  2  0  1  1
 0  1  0  1  1  2  1  1  1  2  0  1  1  1  1  1  0  1  2  0
 0  1  0  0  0  2  1  2  1  1  1  0  1  0  1  1  1  1  1  1
 0  1  1  0  2  0  2  0  0  1  1  0  2  2  2  2  1  2  2  1
 1  0  1  0  2  1  1  0  0  2  2  2  0  2  2  0  1  1  1  1
 1  1  1  2  1  1  1  1  0  1  1  1  1  1  1  1  1  1  1  1
 2  1  2  1  1  2  2  0  1  2  1  0  2  2  2  1  1  1  1  1
 2  2  1  2  1  0  2  1  0  1  1  1  1  1  1  1  0  1  1  0

Select rows where row sum is > 19

In [37]:
vec(sum(M,dims=2) .> 19)

10-element BitVector:
 0
 0
 0
 0
 1
 0
 1
 1
 1
 1

In [38]:
M[vec(sum(M,dims=2) .> 19), :]

5×20 Matrix{Int64}:
 2  1  2  1  1  2  2  0  1  2  1  0  2  2  2  1  1  1  1  1
 2  2  1  2  1  0  2  1  0  1  1  1  1  1  1  1  0  1  1  0
 0  1  1  0  2  0  2  0  0  1  1  0  2  2  2  2  1  2  2  1
 1  0  1  0  2  1  1  0  0  2  2  2  0  2  2  0  1  1  1  1
 1  1  1  2  1  1  1  1  0  1  1  1  1  1  1  1  1  1  1  1

Get random subset of 5 columns from M

In [ ]:
sel = sample(1:size(M,2),5,replace=false)
M[:,sel]